# Análisis integral de conjunto de datos para la industria y creación de un panel de visualización y despliegue de modelos

El objetivo de este notebook es desarrollar la aplicación interactiva de analítica industrial que tenga 
visualización, modelado y despliegue de inteligencia artificial. El dataset seleccionado para ello es Tennessee Eastman Process Simulation Dataset, una simulación de un proceso químico industrial, diseñado como benchmark para estudiar control, detección de fallos y monitoreo de procesos. Por lo que incluye datos de operación normal y con múltiples fallos simulados, permitiendo evaluar algoritmos de diagnóstico y predicción en entornos realistas.

## Preparación de Datos

Empezamos con la carga, exploración, limpieza y preprocesamiento de los datos, dejando el dataset listo para las siguientes implementaciones. 

In [4]:
# Importación de librerías
# !pip install pyreadr
!pip install scikit-learn

# import pyreadr
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

### 1. Función de carga de datos

Los archivos originales estaban en formato .RData. Asi que para leerlos teniamos que utilizar la librería **pyreadr**, que devuelve un diccionario con los objetos contenidos en el archivo. 

Como el dataset con el que vamos a trabajar es el dataset reducido y el formato de los archivos ahora es **.csv**, vamos a definir una función que carga el archivo y devuelve el DataFrame correspondiente.

In [2]:
# def load_rdata(file_path):
#     result = pyreadr.read_r(file_path)
#     df_name = list(result.keys())[0]
#     return result[df_name]

In [5]:
DATA_DIR = "DatasetReducido"

def load_csv(file_name):
    file_path = os.path.join(DATA_DIR, file_name)
    return pd.read_csv(file_path)

### 2. Carga de datasets

El benchmark TEP incluye cuatro conjuntos principales:

- **Fault Free Training**: simulaciones sin fallos para entrenamiento.  
- **Fault Free Testing**: simulaciones sin fallos para validación.  
- **Faulty Training**: simulaciones con distintos fallos para entrenamiento.  
- **Faulty Testing**: simulaciones con fallos para validación.

In [6]:
datasets = {
    "Fault Free Training": load_csv("FaultFree_Training_reduced.csv"),
    "Fault Free Testing": load_csv("FaultFree_Testing_reduced.csv"),
    "Faulty Training": load_csv("Faulty_Training_reduced.csv"),
    "Faulty Testing": load_csv("Faulty_Testing_reduced.csv")
}

In [11]:
for name, df in datasets.items():
    print(f"\n--- {name} ---")
    print("Columnas:", df.columns.tolist())
    print("Tamaño:", df.shape)
    print("Primeras filas:")
    display(df.head())


--- Fault Free Training ---
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']
Tamaño: (10000, 56)
Primeras filas:


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,0.0,10.0,1,0.25125,3671.5,4498.3,9.3137,26.652,42.276,2705.1,...,24.572,60.716,22.456,39.745,41.519,44.748,47.380,41.004,18.549,0
1,0.0,10.0,2,0.25068,3661.1,4533.6,9.2646,26.894,42.133,2705.6,...,24.629,62.093,22.091,40.031,35.268,49.621,47.300,41.822,14.796,0
2,0.0,10.0,3,0.24835,3656.7,4496.2,9.3947,27.098,42.706,2704.4,...,24.865,60.299,22.107,39.766,39.829,47.063,47.545,40.894,18.325,0
3,0.0,10.0,4,0.24777,3649.6,4524.2,9.4125,26.842,42.766,2704.5,...,24.922,59.825,22.018,39.975,39.998,46.470,47.433,41.288,15.854,0
4,0.0,10.0,5,0.26755,3587.1,4476.1,9.3137,27.090,42.104,2703.6,...,26.065,59.509,22.012,40.246,35.800,45.888,47.480,40.297,17.302,0



--- Fault Free Testing ---
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']
Tamaño: (19200, 56)
Primeras filas:


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,0,10.0,1,0.25122,3670.3,4469.7,9.3254,26.859,42.366,2704.2,...,24.575,62.416,21.959,39.931,35.665,42.923,47.299,41.490,18.261,0
1,0,10.0,2,0.25194,3651.0,4450.6,9.1660,26.618,42.044,2705.3,...,24.504,64.163,22.247,39.968,41.853,47.886,47.260,41.486,18.260,0
2,0,10.0,3,0.25131,3646.0,4508.5,9.3401,26.596,42.780,2705.3,...,24.574,62.551,22.187,39.775,39.963,47.403,47.516,40.858,20.620,0
3,0,10.0,4,0.25118,3627.0,4516.3,9.2717,26.864,42.387,2705.6,...,24.587,61.818,22.192,39.664,36.950,46.967,47.496,41.120,16.733,0
4,0,10.0,5,0.25691,3656.9,4515.5,9.3580,27.041,42.763,2704.3,...,25.663,61.038,22.198,39.918,42.758,43.768,47.433,40.575,18.091,0



--- Faulty Training ---
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']
Tamaño: (100000, 56)
Primeras filas:


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,1,69.0,1,0.25105,3653.8,4502.8,9.3635,26.643,42.327,2704.7,...,24.592,62.980,22.080,40.112,37.428,44.084,47.385,42.109,15.128,1
1,1,69.0,2,0.25185,3689.2,4479.0,9.2082,26.993,42.043,2706.0,...,24.514,61.301,22.193,39.931,37.877,44.949,47.455,40.861,17.614,1
2,1,69.0,3,0.25221,3667.5,4555.3,9.3165,26.827,42.402,2704.3,...,24.486,60.529,21.828,40.058,33.198,44.813,47.457,41.130,20.663,1
3,1,69.0,4,0.25256,3673.3,4486.1,9.3145,26.985,42.461,2701.0,...,24.451,63.898,21.653,39.950,37.459,49.350,47.511,40.586,19.034,1
4,1,69.0,5,0.24168,3708.1,4508.5,9.3236,27.199,42.194,2703.8,...,23.656,62.561,22.162,37.875,35.252,47.723,47.532,41.481,17.127,1



--- Faulty Testing ---
Columnas: ['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11', 'fault_present']
Tamaño: (192000, 56)
Primeras filas:


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,1,69.0,1,0.25065,3636.6,4496.9,9.3156,26.715,42.248,2705.0,...,24.631,60.583,22.172,40.228,35.160,47.077,47.501,40.349,18.834,1
1,1,69.0,2,0.24807,3675.0,4498.3,9.3146,27.204,42.252,2704.1,...,24.884,61.844,22.375,40.128,38.003,48.149,47.486,40.967,20.043,1
2,1,69.0,3,0.24843,3629.2,4504.9,9.5091,26.875,42.140,2705.3,...,24.858,62.711,22.196,40.087,32.950,47.967,47.339,41.539,17.325,1
3,1,69.0,4,0.25042,3678.2,4512.0,9.2943,26.884,42.065,2705.8,...,24.662,62.549,22.183,40.184,40.418,45.045,47.446,41.616,17.050,1
4,1,69.0,5,0.26560,3703.6,4496.5,9.3821,26.933,42.227,2706.3,...,25.898,61.799,22.258,41.218,34.065,46.775,47.492,41.415,17.790,1


### 3. Definición de variables

El proceso TEP incluyen dos grupos de variables, que vamos a definir para facilitar su acceso cuando sea necesario. 

- 41 variables de proceso, mediciones de sensores dentro de la planta química simulada: (`xmeas_1` a `xmeas_41`)  
- 11 variables de actuadores, variables manipuladas por los controladores de la planta : (`xmv_1` a `xmv_11`)

In [7]:
process_vars = [f"xmeas_{i}" for i in range(1, 42)]
actuator_vars = [f"xmv_{i}" for i in range(1, 12)]

In [10]:
print("\nVariables de proceso:", process_vars)
print("\nVariables de actuadores:", actuator_vars)


Variables de proceso: ['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41']

Variables de actuadores: ['xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11']


### 4. Información general de cada dataset

Enseñamos en formato DataFrame:
- Tamaño del dataset (df.shape)
- Número de simulaciones (simulationRun.nunique())
- Rango temporal de muestras (sample.min() – sample.max())
- Fallos presentes (si existe la columna faultNumber)
- Conteo de fallos (solo en los datasets Faulty)

In [12]:
import pandas as pd

# Crear lista de resúmenes
resumen = []
for name, df in datasets.items():
    info = {
        "Dataset": name,
        "Tamaño": df.shape,
        "Nº simulaciones": df['simulationRun'].nunique(),
        "Rango muestras": f"{df['sample'].min()} - {df['sample'].max()}"
    }
    if 'faultNumber' in df.columns:
        info["Fallos presentes"] = sorted(df['faultNumber'].unique())
        if name.startswith("Faulty"):
            info["Conteo fallos"] = df['faultNumber'].value_counts().sort_index().to_dict()
    resumen.append(info)

# Convertir a DataFrame
tabla_resumen = pd.DataFrame(resumen)

# Mostrar tabla resumen
print("\n=== Resumen de datasets ===")
display(tabla_resumen)

# Mostrar head de cada dataset
for name, df in datasets.items():
    print(f"\n--- Head de {name} ---")
    display(df.head())


=== Resumen de datasets ===


,Dataset,Tamaño,Nº simulaciones,Rango muestras,Fallos presentes,Conteo fallos
0,Fault Free Training,"(10000, 56)",20,1 - 500,[0.0],NaN
1,Fault Free Testing,"(19200, 56)",20,1 - 960,[0],NaN
2,Faulty Training,"(100000, 56)",10,1 - 500,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, ..."
3,Faulty Testing,"(192000, 56)",10,1 - 960,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1: 9600, 2: 9600, 3: 9600, 4: 9600, 5: 9600, ..."



--- Head de Fault Free Training ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,0.0,10.0,1,0.25125,3671.5,4498.3,9.3137,26.652,42.276,2705.1,...,24.572,60.716,22.456,39.745,41.519,44.748,47.380,41.004,18.549,0
1,0.0,10.0,2,0.25068,3661.1,4533.6,9.2646,26.894,42.133,2705.6,...,24.629,62.093,22.091,40.031,35.268,49.621,47.300,41.822,14.796,0
2,0.0,10.0,3,0.24835,3656.7,4496.2,9.3947,27.098,42.706,2704.4,...,24.865,60.299,22.107,39.766,39.829,47.063,47.545,40.894,18.325,0
3,0.0,10.0,4,0.24777,3649.6,4524.2,9.4125,26.842,42.766,2704.5,...,24.922,59.825,22.018,39.975,39.998,46.470,47.433,41.288,15.854,0
4,0.0,10.0,5,0.26755,3587.1,4476.1,9.3137,27.090,42.104,2703.6,...,26.065,59.509,22.012,40.246,35.800,45.888,47.480,40.297,17.302,0



--- Head de Fault Free Testing ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,0,10.0,1,0.25122,3670.3,4469.7,9.3254,26.859,42.366,2704.2,...,24.575,62.416,21.959,39.931,35.665,42.923,47.299,41.490,18.261,0
1,0,10.0,2,0.25194,3651.0,4450.6,9.1660,26.618,42.044,2705.3,...,24.504,64.163,22.247,39.968,41.853,47.886,47.260,41.486,18.260,0
2,0,10.0,3,0.25131,3646.0,4508.5,9.3401,26.596,42.780,2705.3,...,24.574,62.551,22.187,39.775,39.963,47.403,47.516,40.858,20.620,0
3,0,10.0,4,0.25118,3627.0,4516.3,9.2717,26.864,42.387,2705.6,...,24.587,61.818,22.192,39.664,36.950,46.967,47.496,41.120,16.733,0
4,0,10.0,5,0.25691,3656.9,4515.5,9.3580,27.041,42.763,2704.3,...,25.663,61.038,22.198,39.918,42.758,43.768,47.433,40.575,18.091,0



--- Head de Faulty Training ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,1,69.0,1,0.25105,3653.8,4502.8,9.3635,26.643,42.327,2704.7,...,24.592,62.980,22.080,40.112,37.428,44.084,47.385,42.109,15.128,1
1,1,69.0,2,0.25185,3689.2,4479.0,9.2082,26.993,42.043,2706.0,...,24.514,61.301,22.193,39.931,37.877,44.949,47.455,40.861,17.614,1
2,1,69.0,3,0.25221,3667.5,4555.3,9.3165,26.827,42.402,2704.3,...,24.486,60.529,21.828,40.058,33.198,44.813,47.457,41.130,20.663,1
3,1,69.0,4,0.25256,3673.3,4486.1,9.3145,26.985,42.461,2701.0,...,24.451,63.898,21.653,39.950,37.459,49.350,47.511,40.586,19.034,1
4,1,69.0,5,0.24168,3708.1,4508.5,9.3236,27.199,42.194,2703.8,...,23.656,62.561,22.162,37.875,35.252,47.723,47.532,41.481,17.127,1



--- Head de Faulty Testing ---


,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,fault_present
0,1,69.0,1,0.25065,3636.6,4496.9,9.3156,26.715,42.248,2705.0,...,24.631,60.583,22.172,40.228,35.160,47.077,47.501,40.349,18.834,1
1,1,69.0,2,0.24807,3675.0,4498.3,9.3146,27.204,42.252,2704.1,...,24.884,61.844,22.375,40.128,38.003,48.149,47.486,40.967,20.043,1
2,1,69.0,3,0.24843,3629.2,4504.9,9.5091,26.875,42.140,2705.3,...,24.858,62.711,22.196,40.087,32.950,47.967,47.339,41.539,17.325,1
3,1,69.0,4,0.25042,3678.2,4512.0,9.2943,26.884,42.065,2705.8,...,24.662,62.549,22.183,40.184,40.418,45.045,47.446,41.616,17.050,1
4,1,69.0,5,0.26560,3703.6,4496.5,9.3821,26.933,42.227,2706.3,...,25.898,61.799,22.258,41.218,34.065,46.775,47.492,41.415,17.790,1


### 5. Estadísticas descriptivas generales

Calculamos estadísticas básicas (media, desviación estándar, mínimo, percentiles y máximo) para todas las variables de proceso y actuadores en cada dataset, mostrando el resultado en una tabla para cada dataset.


In [13]:
import pandas as pd

# Crear lista de resúmenes estadísticos
estadisticas = []
for name, df in datasets.items():
    stats = df[process_vars + actuator_vars].describe().T[['mean','std','min','25%','50%','75%','max']]
    stats.insert(0, "Variable", stats.index)  
    stats.insert(0, "Dataset", name)          
    estadisticas.append(stats)

# Concatenar todas las tablas en una sola
tabla_estadisticas = pd.concat(estadisticas, ignore_index=True)

# Mostrar tabla resumen
print("\n=== Estadísticas generales de todos los datasets ===")
display(tabla_estadisticas)

# Mostrar head de cada dataset
for name, df in datasets.items():
    print(f"\n--- Head de {name} ---")
    display(df[process_vars + actuator_vars].head())


=== Estadísticas generales de todos los datasets ===


,Dataset,Variable,mean,std,min,25%,50%,75%,max
0,Fault Free Training,xmeas_1,0.250457,0.032023,0.131300,0.22930,0.25098,0.272402,0.38741
1,Fault Free Training,xmeas_2,3664.518500,34.592224,3533.600000,3641.70000,3664.30000,3687.625000,3782.40000
2,Fault Free Training,xmeas_3,4507.716210,39.418970,4358.700000,4481.80000,4507.40000,4534.100000,4683.70000
3,Fault Free Training,xmeas_4,9.346311,0.088194,8.984500,9.28760,9.34750,9.405925,9.69770
4,Fault Free Training,xmeas_5,26.902388,0.212431,26.184000,26.75800,26.90000,27.048000,27.74200
...,...,...,...,...,...,...,...,...,...
203,Faulty Testing,xmv_7,38.071736,2.969804,24.811000,35.98600,38.14300,39.994000,51.43300
204,Faulty Testing,xmv_8,46.419787,2.353492,36.814000,44.81600,46.34300,48.021000,57.37700
205,Faulty Testing,xmv_9,49.916307,17.487117,-0.606600,45.64075,48.10000,51.309000,100.50000
206,Faulty Testing,xmv_10,41.779310,11.680778,-0.229890,40.59800,41.18400,41.841000,100.46000



--- Head de Fault Free Training ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25125,3671.5,4498.3,9.3137,26.652,42.276,2705.1,74.331,120.42,0.33973,...,54.112,24.572,60.716,22.456,39.745,41.519,44.748,47.380,41.004,18.549
1,0.25068,3661.1,4533.6,9.2646,26.894,42.133,2705.6,75.509,120.41,0.33739,...,53.689,24.629,62.093,22.091,40.031,35.268,49.621,47.300,41.822,14.796
2,0.24835,3656.7,4496.2,9.3947,27.098,42.706,2704.4,74.321,120.41,0.33955,...,54.136,24.865,60.299,22.107,39.766,39.829,47.063,47.545,40.894,18.325
3,0.24777,3649.6,4524.2,9.4125,26.842,42.766,2704.5,75.079,120.40,0.33784,...,53.801,24.922,59.825,22.018,39.975,39.998,46.470,47.433,41.288,15.854
4,0.26755,3587.1,4476.1,9.3137,27.090,42.104,2703.6,74.589,120.39,0.34044,...,54.716,26.065,59.509,22.012,40.246,35.800,45.888,47.480,40.297,17.302



--- Head de Fault Free Testing ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25122,3670.3,4469.7,9.3254,26.859,42.366,2704.2,74.405,120.41,0.33821,...,54.454,24.575,62.416,21.959,39.931,35.665,42.923,47.299,41.490,18.261
1,0.25194,3651.0,4450.6,9.1660,26.618,42.044,2705.3,73.882,120.41,0.33791,...,54.683,24.504,64.163,22.247,39.968,41.853,47.886,47.260,41.486,18.260
2,0.25131,3646.0,4508.5,9.3401,26.596,42.780,2705.3,74.760,120.40,0.33948,...,53.988,24.574,62.551,22.187,39.775,39.963,47.403,47.516,40.858,20.620
3,0.25118,3627.0,4516.3,9.2717,26.864,42.387,2705.6,75.117,120.40,0.34038,...,53.895,24.587,61.818,22.192,39.664,36.950,46.967,47.496,41.120,16.733
4,0.25691,3656.9,4515.5,9.3580,27.041,42.763,2704.3,75.001,120.41,0.33484,...,53.395,25.663,61.038,22.198,39.918,42.758,43.768,47.433,40.575,18.091



--- Head de Faulty Training ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25105,3653.8,4502.8,9.3635,26.643,42.327,2704.7,75.130,120.42,0.33673,...,54.058,24.592,62.980,22.080,40.112,37.428,44.084,47.385,42.109,15.128
1,0.25185,3689.2,4479.0,9.2082,26.993,42.043,2706.0,75.619,120.39,0.33821,...,54.342,24.514,61.301,22.193,39.931,37.877,44.949,47.455,40.861,17.614
2,0.25221,3667.5,4555.3,9.3165,26.827,42.402,2704.3,74.716,120.39,0.33716,...,53.428,24.486,60.529,21.828,40.058,33.198,44.813,47.457,41.130,20.663
3,0.25256,3673.3,4486.1,9.3145,26.985,42.461,2701.0,74.874,120.40,0.33804,...,54.257,24.451,63.898,21.653,39.950,37.459,49.350,47.511,40.586,19.034
4,0.24168,3708.1,4508.5,9.3236,27.199,42.194,2703.8,74.546,120.41,0.32060,...,54.229,23.656,62.561,22.162,37.875,35.252,47.723,47.532,41.481,17.127



--- Head de Faulty Testing ---


,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.25065,3636.6,4496.9,9.3156,26.715,42.248,2705.0,75.042,120.40,0.33578,...,54.128,24.631,60.583,22.172,40.228,35.160,47.077,47.501,40.349,18.834
1,0.24807,3675.0,4498.3,9.3146,27.204,42.252,2704.1,74.442,120.38,0.33660,...,54.112,24.884,61.844,22.375,40.128,38.003,48.149,47.486,40.967,20.043
2,0.24843,3629.2,4504.9,9.5091,26.875,42.140,2705.3,75.045,120.40,0.33692,...,54.031,24.858,62.711,22.196,40.087,32.950,47.967,47.339,41.539,17.325
3,0.25042,3678.2,4512.0,9.2943,26.884,42.065,2705.8,75.269,120.39,0.33613,...,53.947,24.662,62.549,22.183,40.184,40.418,45.045,47.446,41.616,17.050
4,0.26560,3703.6,4496.5,9.3821,26.933,42.227,2706.3,74.549,120.42,0.33894,...,53.441,25.898,61.799,22.258,41.218,34.065,46.775,47.492,41.415,17.790


### 6. Tabla resumen de simulaciones y fallos

Sintetizamos la información clave de cada dataset en una tabla.

In [14]:
summary = []
for name, df in datasets.items():
    entry = {
        "Dataset": name,
        "Simulaciones": df['simulationRun'].nunique(),
        "Muestras": df['sample'].nunique()
    }
    if 'faultNumber' in df.columns:
        entry["Fallos únicos"] = df['faultNumber'].nunique()
    else:
        entry["Fallos únicos"] = 0
    summary.append(entry)

summary_df = pd.DataFrame(summary)
summary_df

,Dataset,Simulaciones,Muestras,Fallos únicos
0,Fault Free Training,20,500,1
1,Fault Free Testing,20,960,1
2,Faulty Training,10,500,20
3,Faulty Testing,10,960,20


### 7. Preprocesamiento: Normalización y variables derivadas

Normalizamos los datos y creamos variables derivadas. Un ejemplo de esto puede ser lo siguiente:
- **Normalización** con `StandardScaler`. Con esto aseguramos que todas las variables estén en la misma escala, algo crítico para algoritmos basados en distancia o redes neuronales. 
- **Variables derivadas**: medias móviles y diferencias temporales. Con lo que demostramos lo significativo que puede ser para dataset con información de tendencias y dinámicas, sin alterar la estructura original.

In [15]:
preprocessed_datasets = {}
scaler = StandardScaler()

for name, df in datasets.items():
    df_copy = df.copy()
    
    # Normalización
    df_copy[process_vars + actuator_vars] = scaler.fit_transform(df_copy[process_vars + actuator_vars])
    
    # Variables derivadas (ejemplo con primeras 5 variables)
    for var in process_vars[:5]:
        df_copy[f"{var}_ma5"] = df_copy[var].rolling(window=5).mean()
        df_copy[f"{var}_diff"] = df_copy[var].diff()
    
    preprocessed_datasets[name] = df_copy

preprocessed_datasets["Fault Free Training"].head(10)

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmeas_1_ma5,xmeas_1_diff,xmeas_2_ma5,xmeas_2_diff,xmeas_3_ma5,xmeas_3_diff,xmeas_4_ma5,xmeas_4_diff,xmeas_5_ma5,xmeas_5_diff
0,0.0,10.0,1,0.024764,0.201833,-0.238887,-0.369778,-1.178736,-0.265085,-0.001227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,10.0,2,0.006964,-0.098828,0.656666,-0.926535,-0.039486,-0.916628,0.062259,...,NaN,-0.017801,NaN,-0.300661,NaN,0.895553,NaN,-0.556757,NaN,1.139251
2,0.0,10.0,3,-0.065800,-0.226030,-0.292164,0.548700,0.920874,1.694102,-0.090106,...,NaN,-0.072764,NaN,-0.127203,NaN,-0.948829,NaN,1.475235,NaN,0.960360
3,0.0,10.0,4,-0.083913,-0.431289,0.418190,0.750539,-0.284283,1.967477,-0.077409,...,NaN,-0.018113,NaN,-0.205259,NaN,0.710353,NaN,0.201838,NaN,-1.205158
4,0.0,10.0,5,0.533800,-2.238144,-0.802096,-0.369778,0.883213,-1.048759,-0.191683,...,0.083163,0.617713,-0.558492,-1.806855,-0.051658,-1.220286,-0.073371,-1.120317,0.060316,1.167496
5,0.0,10.0,6,0.460411,-2.475203,0.890068,-0.036405,0.732568,-0.310647,0.201927,...,0.170292,-0.073389,-1.093899,-0.237059,0.174133,1.692163,-0.006696,0.333374,0.442577,-0.150645
6,0.0,10.0,7,0.238372,0.175814,0.212695,0.031631,0.266511,-1.463378,0.532050,...,0.216574,-0.222039,-1.038970,2.651017,0.085339,-0.677373,0.184937,0.068035,0.503777,-0.466057
7,0.0,10.0,8,0.274285,-0.757968,0.603389,0.647352,-1.884883,0.883090,0.493959,...,0.284591,0.035914,-1.145358,-0.933783,0.264449,0.390694,0.204668,0.615721,-0.057375,-2.151395
8,0.0,10.0,9,-0.874011,-1.061520,0.068087,-0.853964,-0.133639,-0.342541,0.176532,...,0.126571,-1.148296,-1.271404,-0.303552,0.194429,-0.535302,-0.116233,-1.501316,-0.027246,1.751245
9,0.0,10.0,10,-0.890250,-1.246542,0.796199,-0.758714,-0.180715,0.236103,0.087653,...,-0.158238,-0.016239,-1.073084,-0.185022,0.514088,0.728112,-0.194020,0.095250,-0.240032,-0.047076


## Análisis exploratorio

En esta sección vamos a realizar un analisis más profundo para poder encontrar relaciones entre variables. Haremos uso de estadística descriptiva y gráficos como herramientas.

In [17]:
# Instalación de librería para leer archivos RData
!pip install pyreadr pandas numpy
!pip install seaborn scikit-learn

# import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Configuración de visualizaciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


### 1.Preparación de Datos y Verificación
Se verifica la estructura, tipos de datos y la ausencia de valores nulos (confirmando el análisis inicial).

In [18]:
# 1. Distribución de faultNumber (Multi-clase)
plt.figure(figsize=(10, 5))
sns.countplot(x='faultNumber', data=df_train_raw)
plt.title('Distribución de Frecuencia por Tipo de Fallo en Entrenamiento (0 = Normal)')
plt.show()

# 2. Distribución Binaria (Fallo vs. Normal)
fault_counts = df_train_raw['fault_present'].value_counts()
plt.figure(figsize=(4, 4))
fault_counts.plot.pie(
    autopct='%1.1f%%', 
    labels=['Fallo (1)', 'Normal (0)'] if fault_counts.iloc[0] == 1 else ['Normal (0)', 'Fallo (1)'], 
    colors=['#FF5722', '#4CAF50']
)
plt.ylabel('')
plt.title('Proporción de Muestras en Entrenamiento: Normal vs. Fallo')
plt.show()

NameError: name 'df_train_raw' is not defined

<Figure size 1000x500 with 0 Axes>

La distribución de faultNumber muestra que todas las clases de fallo están representadas, que sugieren una simulación balanceada.

A nivel binario, el conjunto de entrenamiento presenta una fuerte desbalance, con un 95.2% de muestras normales y solo 4.8% con fallo, lo que indica que será necesario aplicar técnicas para manejar el desbalance al entrenar modelos de clasificación.

### 2. Análisis Exploratorio de Datos (EDA)
El EDA se realizará principalmente sobre el conjunto de entrenamiento para entender la dinámica del proceso.

In [ ]:
# Seleccionar solo variables de proceso/manipuladas
process_vars = [col for col in df_train_raw.columns if 'xmeas' in col or 'xmv' in col]
cols_to_corr = process_vars + ['fault_present']

corr_matrix = df_train_raw[cols_to_corr].corr()

# Mostrar Heatmap de correlación (limitado a las 10 variables más correlacionadas con el fallo)
high_corr_features = corr_matrix['fault_present'].abs().sort_values(ascending=False).index[:11]
plt.figure(figsize=(10, 8))
sns.heatmap(
    df_train_raw[high_corr_features].corr(), 
    annot=True, 
    fmt=".2f", 
    cmap='RdBu_r'
)
plt.title('Mapa de Calor de Correlación (Top 10 Features vs. Fallo) - Entrenamiento')
plt.show()

# Conclusión: Se identifican los predictores más fuertes para la fase de modelado.

Las correlaciones con la variable fault_present son muy bajas, indicando que ningún sensor o variable manipulada presenta una relación lineal fuerte con la presencia de fallos. Por otro lado, si se muestra correlación entre otras variables, lo cual puede indicar redundancia en las mediciones. Para el proyecto, esto quiere decir que podemos aplicar selección de variables o reducción de dimensiones durante el modelado.

### 2.2. Análisis de Series Temporales (Ejemplo)

In [ ]:
# Seleccionar una simulación con un fallo conocido (ejemplo: la primera simulación con fallo)
simulation_run = df_train_raw[df_train_raw['fault_present'] == 1]['simulationRun'].iloc[0]
df_sim_fault = df_train_raw[df_train_raw['simulationRun'] == simulation_run].copy()

# Variable (ejemplo de xmeas_1)
key_variable = 'xmeas_1'

plt.figure(figsize=(12, 4))
sns.lineplot(x='sample', y=key_variable, data=df_sim_fault, hue='fault_present', style='fault_present', markers=False)

# Marcar el inicio de la inyección del fallo
fault_start_sample = df_sim_fault[df_sim_fault['fault_present'] == 1]['sample'].min()
if pd.notna(fault_start_sample):
    plt.axvline(x=fault_start_sample, color='r', linestyle='--', label='Inicio del Fallo')

plt.title(f'Serie Temporal de {key_variable} en Simulación {simulation_run} (Entrenamiento)')
plt.legend()
plt.show()

Aqui podemos ver que la variable xmeans_1 tiene un alto nivel de ruido, pero el valor de la clase fallo tiene valores ligeramente superiores.

El rango de variación del fallo (la sombra naranja) es mucho más amplio que la clase normal, lo que indica un aumento en la desviación e inestabilidad del proceso después del comienzo del fallo.